In [9]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import datetime
from datetime import date
import time
from ipywidgets import AppLayout, Button
from IPython.display import HTML, display, Markdown
from bqplot import pyplot as plt
import ipyvuetify as v
import ipywidgets as widgets
from traitlets import Unicode, List

In [2]:
# General Variables
today = date.today()
fdate = date.today().strftime("%m-%d-%Y")
time = time.strftime("%H:%M:%S")
# Input/Storage
INPUT = "../Data/Input/"
OUTPUT = "../Data/Output/"
GRAPHS = "../Data/Graphs/"

# Kaleida Health - COVID 19 Daily Report
<hr>

In [3]:
# Global Data
# URL
# 1 Request URL
url = 'https://www.worldometers.info/coronavirus/'
page = requests.get(url)
# 2 Parse HTML content
soup = BeautifulSoup(page.text, 'html.parser')
# 3 Store the data
data = {}
#Get the table having the class country table
table = soup.find("table", attrs={"id": "main_table_countries"})
table_data = table.tbody.find_all("tr")  
# Get data into table
df = []
for i in range(0,len(table_data)):
    for td in table_data[i].find_all("td"):
        # remove any newlines and extra spaces from left and right
        df.append(td.text.replace('\n', ' ').strip())


countries = pd.DataFrame([])
for i in range(0, len(df), 9):
    countries = countries.append(pd.DataFrame({'Country/Other': df[i], 'Total Cases': df[i+1], 'New Cases':df[i+2],
                                              'Total Deaths':df[i+3], 'New Deaths':df[i+4], 'Rate 1M Pop':df[i+8]},
                                              index =[0]), ignore_index=True)

# Data Modifications and calculations
# Columns to convert to string
cols = ['Total Cases', 'New Cases', 'Total Deaths', 'New Deaths', 'Rate 1M Pop']
# Replace + and , for numeric values
countries = countries.applymap(lambda x: x.replace('+', ''))
countries = countries.applymap(lambda x: x.replace(',', ''))
# Convert to numeric
countries[cols] = countries[cols].apply(pd.to_numeric, errors='coerce', axis=1)
# Add mortality rate column
countries['Mor_Rate'] = round((countries['Total Deaths']/countries['Total Cases'])*100,2)
# Calculate mean mortality rate on top affected countries
df1 = countries[0:9]
mean_MR = np.mean(df1['Mor_Rate'])

# Output data per day
#countries.to_csv(OUTPUT+'cases_world'+fdate+'.csv', index=False)

In [4]:
# Extract data for US
# URL
# 1 Request URL
url = 'https://www.cdc.gov/coronavirus/2019-ncov/cases-in-us.html'
page = requests.get(url)
# 2 Parse HTML content
soup = BeautifulSoup(page.text, 'html.parser')
# 3 Extract cases data
cdc_data = soup.find_all(attrs={"class": "card-body bg-white"})

# Create dataset of extracted data
df = []
for ul in cdc_data:
    for li in ul.find_all('li'):
        df.append(li.text.replace('\n', ' ').strip())

        
# US specific cases - CDC
cases_us = df[0].split(': ')
# Replace + and , for numeric values
cases_us = int(cases_us[1].replace(',', ''))

# Total US deaths - CDC
deaths_us = df[1].split(': ')
deaths_us = pd.to_numeric(deaths_us[1])
# Calculate mortality rate
us_MR = round((deaths_us/cases_us)*100,2)

# Create table
data = {'Cases': [cases_us],
       'Deaths': [deaths_us],
       'Mortality Rate': [us_MR]}
us_data = pd.DataFrame(data)

# Output Daily Data
#us_data.to_csv(OUTPUT+'cases_us'+fdate+'.csv', index=False)

In [6]:
# Extract data for NY State cases
# URL
# 1 Request URL
url = 'https://www.health.ny.gov/diseases/communicable/coronavirus'
page = requests.get(url)
# 2 Parse HTML content
soup = BeautifulSoup(page.text, 'html.parser')
# 3 Create a data dictionary to store the data
data = {}
#Get the table having the class country table
table = soup.find("table", attrs={"id": "case_count_table"})
table_data = table.find_all("tr")  
# Get all the headings of Lists
df = []
for i in range(0,len(table_data)):
    for td in table_data[i].find_all("td"):
        # remove any newlines and extra spaces from left and right
        df.append(td.text.replace('\n', ' ').strip())
        


counties = pd.DataFrame([])
for i in range(0, len(df), 2):
    counties = counties.append(pd.DataFrame({'County': df[i], 'Cases': df[i+1]},
                                              index =[0]), ignore_index=True)

    
# NY state Modification for Counties and State Tables
nys = counties[-3:]
counties_cases = counties[0:-3]
# Total state cases for print
cases_nys = pd.to_numeric(counties.iloc[-1,1])
#Erie County
erie = counties[counties['County']=='Erie'].reset_index()
cases_erie = pd.to_numeric(erie.Cases[0])

#counties.to_csv(OUTPUT+'cases_nys'+fdate+'.csv', index=False)
#counties_cases.to_csv(OUTPUT+'cases_ny_counties'+fdate+'.csv', index=False)

# Situation
<hr>

In [8]:
display(Markdown('<font size="5"><soft>New York State currently has: {} COVID-19 cases as of {} {}.</soft></font>'.format(cases_nys, time, fdate)))
display(Markdown('<font size="5"><soft>Erie County: {} confirmed cases of COVID-19.</soft></font>'.format(cases_erie)))
print('\n')
# display(Markdown('<font size="3"><soft>Kaleida: 0 Reported/Confirmed cases of COVID-19.</soft></font>'))
# display(Markdown('<font size="3"><soft>ECMC: 0 Reported/Confirmed cases of COVID-19.</soft></font>'))
# display(Markdown('<font size="3"><soft>Catholic Health: 0 Reported/Confirmed cases of COVID-19.</soft></font>')) 
# print('\n')
display(Markdown('<font size="2"><soft>Reported cases as of: {} {}.</soft></font>'.format(time, fdate)))

<font size="5"><soft>New York State currently has: 729 COVID-19 cases as of 15:57:20 03-15-2020.</soft></font>

<font size="5"><soft>Erie County: 3 confirmed cases of COVID-19.</soft></font>

<font size="2"><soft>Reported cases as of: 15:57:20 03-15-2020.</soft></font>

<hr>
# Background
<hr>

In [ ]:
display(Markdown('<font size="4"><soft>Incubation, Exposure and Symptom Development</soft></font>'))
display(Markdown('<font size="3"><soft>2-14 days represents the current official estimated incubation period range.</soft></font>'))
display(Markdown('<font size="3"><soft>The median incubation period has been estimated to be 5.1 days (95% CI, 4.5 - 5.8 days).</soft></font>'))
display(Markdown('<font size="3"><soft>97.5% of those who develop symptoms will do so within 11.5 days (CI, 8.2 - 15.6 days).</soft></font>'))
display(Markdown('<font size="3"><soft>101 out of every 10 000 cases (99th percentile, 482) will develop symptoms after 14 days of active monitoring or quarantine.</soft></font>'))
print('\n')
display(Markdown('<font size="4"><soft>Disease Transmission and Doubling Rate</soft></font>'))
display(Markdown('<font size="3"><soft>Variable range of 3-9 days depending upon the country’s medical infrastructure, geography and population density.</soft></font>'))
print('\n')
display(Markdown('<font size="2"><soft>Data updated: 3/10/2020.</soft></font>'))

<hr>
# Global
<hr>

In [ ]:
# Display values
display(Markdown('<font size="3"><soft>The calculated mean mortality rate for the top 10 affected countries is: {} percent.</soft></font>'.format('%.2f'%mean_MR)))

In [ ]:
# # Table of top 10 affected countries/other
# df1 = countries[0:9]

# fig = go.Figure(data=[go.Table(header=dict(values=['Country/Other', 'Total Cases', 'New Cases', 'Total Deaths',
#                                                    'Rate per 1 Million Population', 'Mortality Rate %']),
#                  cells=dict(values=[df1['Country/Other'],df1['Total Cases'], df1['New Cases'], df1['Total Deaths'],
#                                    df1['Rate 1M Pop'], df1['Mor_Rate']]))
#                      ])

# fig.update_layout(
#     title='Countries with highest cases of COVID-19')
# fig.update_layout(height=425)
# fig.show()

In [ ]:
# Table of all countries with mortality rate
class MyDataTable(v.VuetifyTemplate):
    headers = List([
        {'text': 'Country', 'sortable':  True, 'value': 'Country/Other', 'align':'left'},
        {'text': 'Total Cases', 'sortable':  True, 'value': 'Total Cases', 'align':'right'},
        {'text': 'New Cases', 'sortable':  True, 'value': 'New Cases', 'align':'right'},
        {'text': 'Total Deaths', 'sortable':  True, 'value': 'Total Deaths', 'align':'right'},
        {'text': 'New Deaths', 'sortable':  True, 'value': 'New Deaths', 'align':'right'},
        {'text': 'Rate per 1 Million Population', 'sortable':  True, 'value': 'Rate 1M Pop', 'align':'right'},
        {'text': 'Mortality Rate', 'sortable':  True, 'value': 'Mor_Rate', 'align':'right'},
    ]).tag(sync=True)

    items = List(countries.to_dict(orient='records')).tag(sync=True)

    template = Unicode('''
        <v-data-table
            :headers="headers"
            :items="items"
            class="elevation-1"
        >
            <template v-slot:items="props">
                <td>{{ props.item.Name }}</td>
                <td class="text-xs-right">{{ props.item.Country }}</td>
            </template>
        </v-data-table>
    ''').tag(sync=True)

MyDataTable()

In [ ]:
display(Markdown('<font size="2"><soft>Reported cases as of: {} {}.</soft></font>'.format(time, fdate)))
display(Markdown('<font size="2"><soft>Sources:</soft></font>'))
display(Markdown('<font size="2"><soft>https://www.worldometers.info/coronavirus/</soft></font>'))

<hr>
# United States
<hr>

In [ ]:
# General US data

In [ ]:
# # Table of cases in the US
# fig = go.Figure(data=[go.Table(header=dict(values=['Total Cases', 'Total Deaths', 'Mortality Rate %']),
#                  cells=dict(values=[cases_us, deaths_us, us_MR]))
#                      ])

# fig.update_layout(
#     title='Cases of COVID-19 in the United States')
# fig.update_layout(height=250)
# fig.show()

In [ ]:
# US Data Table
class MyDataTable(v.VuetifyTemplate):
    headers = List([
        {'text': 'Total US Cases', 'value': 'Cases', 'align': 'center'},
        {'text': 'Total Deaths', 'value': 'Deaths', 'align': 'center'},
        {'text': 'Mortality Rate', 'value': 'Mortality Rate', 'align': 'center'}
    ]).tag(sync=True)

    items = List(us_data.to_dict(orient='records')).tag(sync=True)

    template = Unicode('''
        <v-data-table
            :headers="headers"
            :items="items"
            class="elevation-1"
        >
            <template v-slot:items="props">
                <td>{{ props.item.Name }}</td>
                <td class="text-xs-right">{{ props.item.Cases }}</td>
            </template>
        </v-data-table>
    ''').tag(sync=True)

MyDataTable()

In [ ]:
display(Markdown('<font size="2"><soft>Reported cases as of: {} {}.</soft></font>'.format(time, fdate)))
display(Markdown('<font size="2"><soft>Sources:</soft></font>'))
display(Markdown('<font size="2"><soft>https://www.cdc.gov/coronavirus/2019-ncov/cases-in-us.html</soft></font>'))

<hr>
# New York State
<hr>

In [ ]:
# NY State General

In [ ]:
# fig = go.Figure(data=[go.Table(header=dict(values=['New York State', 'Cases']),
#                  cells=dict(values=[nys['County'],nys['Cases']]))
#                      ])

# fig.update_layout(
#     title='New York State Cases of COVID-19')

# fig.show()

In [ ]:
class MyDataTable(v.VuetifyTemplate):
    headers = List([
        {'text': 'New York', 'sortable':  True, 'value': 'County'},
        {'text': 'Cases', 'value': 'Cases', 'align': 'right'}
    ]).tag(sync=True)

    items = List(nys.to_dict(orient='records')).tag(sync=True)

    template = Unicode('''
        <v-data-table
            :headers="headers"
            :items="items"
            class="elevation-1"
        >
            <template v-slot:items="props">
                <td>{{ props.item.Name }}</td>
                <td class="text-xs-right">{{ props.item.Cases }}</td>
            </template>
        </v-data-table>
    ''').tag(sync=True)

MyDataTable()

In [ ]:
# Counties

In [ ]:
# fig = go.Figure(data=[go.Table(header=dict(values=['New York State', 'Cases']),
#                  cells=dict(values=[counties_cases['County'],counties_cases['Cases']]))
#                      ])

# fig.update_layout(
#     title='New York Counties Cases of COVID-19')
# fig.show()

In [ ]:
class MyDataTable(v.VuetifyTemplate):
    headers = List([
        {'text': 'County', 'sortable':  True, 'value': 'County'},
        {'text': 'Cases', 'value': 'Cases', 'align': 'right'}
    ]).tag(sync=True)

    items = List(counties_cases.to_dict(orient='records')).tag(sync=True)

    template = Unicode('''
        <v-data-table
            :headers="headers"
            :items="items"
            class="elevation-1"
        >
            <template v-slot:items="props">
                <td>{{ props.item.Name }}</td>
                <td class="text-xs-right">{{ props.item.Cases }}</td>
            </template>
        </v-data-table>
    ''').tag(sync=True)

MyDataTable()

In [ ]:
display(Markdown('<font size="2"><soft>Reported cases as of: {} {}.</soft></font>'.format(time, fdate)))
display(Markdown('<font size="2"><soft>Sources:</soft></font>'))
display(Markdown('<font size="2"><soft>https://www.health.ny.gov/diseases/communicable/coronavirus/</soft></font>'))

<hr>
# Age, Gender and Comorbidities
<hr>

In [ ]:
# # Mortality Rate by age categories
# age_cats = ['80+ years old', '70-79 years old', '60-69 years old', '50-59 years old', '40-49 years old',
#            '30-39 years old', '20-29 years old', '10-19 years old', '0-9 years old']
# deaths_confirmed = ['21.9%']
# deaths_all = ['14.8%', '8.0%', '3.6%', '1.3%', '0.4%', '0.2%', '0.2%', '0.2%', 'No Fatalities']

# fig = go.Figure(data=[go.Table(header=dict(values=['Age', 'Death Rate (Confirmed Cases)', 'Death Rate(All Cases)']),
#                  cells=dict(values=[age_cats, deaths_confirmed, deaths_all]))
#                      ])

# fig.update_layout(
#     title='Fatality Rate by Age Groups')
# fig.update_layout(height=400)
# fig.show()

In [ ]:
# Create table of Age categories
data = {
'age_cats' : ['80+ years old', '70-79 years old', '60-69 years old', '50-59 years old', '40-49 years old',
            '30-39 years old', '20-29 years old', '10-19 years old', '0-9 years old'], 
'deaths_confirmed' : ['21.9%', None, None, None, None, None, None, None, None],
'deaths_all' : ['14.8%', '8.0%', '3.6%', '1.3%', '0.4%', '0.2%', '0.2%', '0.2%', 'No Fatalities']
}
age_data = pd.DataFrame(data)

In [ ]:
# Age- Specific data
class MyDataTable(v.VuetifyTemplate):
    headers = List([
        {'text': 'Age Category', 'value': 'age_cats', 'align': 'center'},
        {'text': 'Deaths (Confirmed Cases)', 'value': 'deaths_confirmed', 'align': 'center'},
        {'text': 'Deaths (All Cases)', 'value': 'deaths_all', 'align': 'center'}
    ]).tag(sync=True)

    items = List(age_data.to_dict(orient='records')).tag(sync=True)

    template = Unicode('''
        <v-data-table
            :headers="headers"
            :items="items"
            class="elevation-1"
        >
            <template v-slot:items="props">
                <td>{{ props.item.Name }}</td>
                <td class="text-xs-right">{{ props.item.age_cats }}</td>
            </template>
        </v-data-table>
    ''').tag(sync=True)

MyDataTable()

In [ ]:
# Create gender table
data={'gender' : ['Male', 'Female'],
'deaths_confirmed' : ['4.7%', '2.8%'],
'deaths_all' : ['2.8%', '1.7%']}
gender_data = pd.DataFrame(data)

In [ ]:
# Gener specific Data

# gender = ['Male', 'Female']
# deaths_confirmed = ['4.7%', '2.8%']
# deaths_all = ['2.8%', '1.7%']

# fig = go.Figure(data=[go.Table(header=dict(values=['Sex', 'Death Rate (Confirmed Cases)', 'Death Rate(All Cases)']),
#                  cells=dict(values=[gender, deaths_confirmed, deaths_all]))
#                      ])

# fig.update_layout(
#     title='Fatality Rate by Sex')
# fig.update_layout(height=250)
# fig.show()

In [ ]:
# Gender table  data
class MyDataTable(v.VuetifyTemplate):
    headers = List([
        {'text': 'Gender', 'value': 'gender', 'align': 'center'},
        {'text': 'Death Rate (Confirmed Cases)', 'value': 'deaths_confirmed', 'align': 'center'},
        {'text': 'Deaths (All Cases)', 'value': 'deaths_all', 'align': 'center'}
    ]).tag(sync=True)

    items = List(gender_data.to_dict(orient='records')).tag(sync=True)

    template = Unicode('''
        <v-data-table
            :headers="headers"
            :items="items"
            class="elevation-1"
        >
            <template v-slot:items="props">
                <td>{{ props.item.Name }}</td>
                <td class="text-xs-right">{{ props.item.age_cats }}</td>
            </template>
        </v-data-table>
    ''').tag(sync=True)

MyDataTable()

In [ ]:
# create table of comorbidities
data={'comor' : ['Cardiovascular Disease', 'Diabetes', 'Chronic Respiratory Disease', 'Hypertension', 'Cancer',
           'No Pre-Existing Conditions'],
'deaths_confirmed' : ['13.2%', '9.2%', '8.0%', '8.4%', '7.6%', None],
'deaths_all' : ['10.5%', '7.3%', '6.3%', '6.0%', '5.6%', '0.9%']
     }
comor_data = pd.DataFrame(data)

In [ ]:
# # Mortality Rate by comorbidities
# comor = ['Cardiovascular Disease', 'Diabetes', 'Chronic Respiratory Disease', 'Hypertension', 'Cancer',
#            'No Pre-Existing Conditions']
# deaths_confirmed = ['13.2%', '9.2%', '8.0%', '8.4%', '7.6%']
# deaths_all = ['10.5%', '7.3%', '6.3%', '6.0%', '5.6%', '0.9%']

# fig = go.Figure(data=[go.Table(header=dict(values=['Pre-Existing Condition', 'Death Rate (Confirmed Cases)', 'Death Rate(All Cases)']),
#                  cells=dict(values=[comor, deaths_confirmed, deaths_all]))
#                      ])

# fig.update_layout(
#     title='Fatality Rate by Comorbidities')
# fig.update_layout(height=500)
# fig.show()

In [ ]:
# Commorbidities Table
class MyDataTable(v.VuetifyTemplate):
    headers = List([
        {'text': 'Pre-existing Condition', 'value': 'comor', 'align': 'center'},
        {'text': 'Death Rate (Confirmed Cases)', 'value': 'deaths_confirmed', 'align': 'center'},
        {'text': 'Deaths (All Cases)', 'value': 'deaths_all', 'align': 'center'}
    ]).tag(sync=True)

    items = List(comor_data.to_dict(orient='records')).tag(sync=True)

    template = Unicode('''
        <v-data-table
            :headers="headers"
            :items="items"
            class="elevation-1"
        >
            <template v-slot:items="props">
                <td>{{ props.item.Name }}</td>
                <td class="text-xs-right">{{ props.item.age_cats }}</td>
            </template>
        </v-data-table>
    ''').tag(sync=True)

MyDataTable()

<hr>
# Cruise Ships Data
<hr>

In [ ]:
# Create dataset
data = {'Cruise Ship':['Diamond Princess', 'World Dream', 'MS Westerdam', 'Grand Princess', 'Nile River'],
        'Date': ['2/5/20-3/1/20', '1/19/20-1/24/20', '2/1/20-2/13/20', '2/11/20-3/9/20', 'null'],
        'Location': ['Yokohoma, Japan', 'Hong Kong, China', 'Sihanoukville, Cambodia', 'Oakland, USA', 'null'],
        'Passengers': [3618, 3800, 650, 3533, 171],
        'Cases': [696, 3, 1, 21, 45],
        'Recovered': [245, None , 1, None , None],
        'Deaths': [7, None, None, 1, None]
       } 
cruise = pd.DataFrame(data)
cruise['Mortality Rate'] = round((cruise['Deaths']/cruise['Cases'])*100,2)

In [ ]:
# fig = go.Figure(data=[go.Table(header=dict(values=['Cruise Ship', 'Location', 'Passengers', 'Cases', 'Recovered',
#                                                   'Deaths', 'Mortality Rate %']),
#                  cells=dict(values=[cruise['Cruise Ship'], cruise['Location'], cruise['Passengers'],
#                                    cruise['Cases'], cruise['Recovered'], cruise['Deaths'], cruise['Mortality Rate']]))
#                      ])

# fig.update_layout(
#     title='Cruise Ships COVID-19 Cases')
# fig.update_layout(height=500)
# fig.show()

In [ ]:
# Cruise data
class MyDataTable(v.VuetifyTemplate):
    headers = List([
        {'text': 'Cruise Ship', 'value': 'Cruise Ship', 'align': 'center'},
        {'text': 'Location', 'value': 'Location', 'align': 'center'},
        {'text': 'Passengers', 'value': 'Passengers', 'align': 'center'},
        {'text': 'Cases', 'value': 'Cases', 'align': 'center'},
        {'text': 'Recovered', 'value': 'Recovered', 'align': 'center'},
        {'text': 'Deaths', 'value': 'Deaths', 'align': 'center'},
        {'text': 'Mortality Rate %', 'value': 'Mortality Rate', 'align': 'center'}
    ]).tag(sync=True)

    items = List(cruise.to_dict(orient='records')).tag(sync=True)

    template = Unicode('''
        <v-data-table
            :headers="headers"
            :items="items"
            class="elevation-1"
        >
            <template v-slot:items="props">
                <td>{{ props.item.Name }}</td>
                <td class="text-xs-right">{{ props.item.age_cats }}</td>
            </template>
        </v-data-table>
    ''').tag(sync=True)

MyDataTable()

Sources
https://www.worldometers.info/coronavirus/coronavirus-incubation-period/
Lauer SA, Grantz KH, Bi Q, et al. The Incubation Period of Coronavirus Disease 2019 (COVID-19) From Publicly Reported Confirmed Cases: Estimation and Application. Ann Intern Med. 2020; [Epub ahead of print 10 March 2020]. doi: https://doi.org/10.7326/M20-0504
http://www.centerforhealthsecurity.org/resources/COVID-19/index.html
http://www.centerforhealthsecurity.org/resources/fact-sheets/pdfs/coronaviruses.pdf'
https://coronavirus.jhu.edu/
https://www.who.int/emergencies/diseases/novel-coronavirus-2019/situation-reports
https://www.worldometers.info/coronavirus/coronavirus-age-sex-demographics/